Na zadanie składają się:

- opracowanie procedury konstruowania problemu programowania matematycznego (Mixed Integer Programing),
którego rozwiązaniem będzie dopuszczalny (pozwalający na uniknięcie konfliktów) zbiór manewrów
statków powietrznych na bazie znanej macierzy konfliktów
- przygotowanie procedury importowania danych testowych z załączonych plików (CM.zip)
- rozwiązanie problemu za pomocą solvera ILOG CPLEX (Python API)
- przygotowanie i przesłanie raportu z kodem źródłowym i wynikami
(zastosowane opcje algorytmu solwera, wybrane rozwiązanie startowe,
 liczba znalezionych rozwiązań dopuszczalnych,
 lista przykładowych dopuszczalnych manewrów statków powietrznych + całkowity czas obliczeń solwera).

# Kod

In [6]:
from docplex.mp.model import Model
import numpy as np
import time

def read_file(filename):
    with open(filename, 'r') as file:
        n = int(filename.split('_')[1].split('=')[1])
        m = int(filename.split('_')[2].split('=')[1].split('.')[0])
        CM = [int(x) for line in file for x in line.split()]
    return n, m, np.array(CM).reshape(n*m, n*m)

def create_chunks(CM, n, m):
    chunks = []
    for i in range(0, n*m, m):
        for j in range(0, n*m, m):
            chunk = CM[i:i+m, j:j+m]
            chunks.append(chunk)
    return chunks

def create_model(n, m):
    model = Model()
    model.name = 'Model_1'
    x = model.binary_var_matrix(range(1, n + 1), range(1, m + 1), name=lambda ns: f'x_{ns[0]}_{ns[1]}')
    return model, x

def add_constraints(model, n, m, x, chunks):
    for i in range(1, n + 1):
        model.add_constraint(model.sum(x[i, j] for j in range(1, m + 1)) == 1)

    for i in range(n):
        for j in range(m):
            for k in range(i + 1, n):
                if i != k:
                    for l in range(m):
                        chunk = chunks[i*n+k]
                        if chunk[j][l] == 1:
                            model.add_constraint(x[i+1, j+1]+x[k+1, l+1] <= 1)

def solve_model(model):
    start_time = time.time()
    model.solve()
    stop_time = time.time()
    return stop_time - start_time

def print_solution(model, execution_time):
    if model.solution is not None:
        print(f'Czas wykonania: {execution_time} sekund')
        print(model.solution.display())
    else:
        print('Nie znaleziono rozwiązania.')

def main():
    tries = 50
    filenames = ['CM_n=10_m=7.txt', 'CM_n=20_m=7.txt', 'CM_n=30_m=7.txt', 'CM_n=40_m=7.txt']
    for filename in filenames:
        print(f'Processing file: {filename}')
        n, m, CM = read_file(filename)
        chunks = create_chunks(CM, n, m)
        model, x = create_model(n, m)
        add_constraints(model, n, m, x, chunks)

        total_time = 0
        for _ in range(tries):
            solution = solve_model(model)
            total_time +=  solution

        average_time = total_time / tries
        print(f'Average execution time for {filename}: {average_time} seconds')
        model.solution.display()
        model.solution.clear()

if __name__ == "__main__":
    main()

Processing file: CM_n=10_m=7.txt
Average execution time for CM_n=10_m=7.txt: 0.0024979257583618163 seconds
solution for: Model_1
status: OPTIMAL_SOLUTION(2)
x_1_1 = 1
x_2_1 = 1
x_3_1 = 1
x_4_3 = 1
x_5_2 = 1
x_6_1 = 1
x_7_1 = 1
x_8_2 = 1
x_9_3 = 1
x_10_1 = 1
Processing file: CM_n=20_m=7.txt
Average execution time for CM_n=20_m=7.txt: 0.0029556083679199217 seconds
solution for: Model_1
status: OPTIMAL_SOLUTION(2)
x_1_6 = 1
x_2_4 = 1
x_3_5 = 1
x_4_4 = 1
x_5_3 = 1
x_6_1 = 1
x_7_1 = 1
x_8_1 = 1
x_9_1 = 1
x_10_2 = 1
x_11_7 = 1
x_12_5 = 1
x_13_5 = 1
x_14_5 = 1
x_15_3 = 1
x_16_4 = 1
x_17_5 = 1
x_18_6 = 1
x_19_4 = 1
x_20_5 = 1
Processing file: CM_n=30_m=7.txt
Average execution time for CM_n=30_m=7.txt: 0.0036261463165283203 seconds
solution for: Model_1
status: OPTIMAL_SOLUTION(2)
x_1_7 = 1
x_2_4 = 1
x_3_1 = 1
x_4_6 = 1
x_5_3 = 1
x_6_6 = 1
x_7_6 = 1
x_8_3 = 1
x_9_4 = 1
x_10_4 = 1
x_11_4 = 1
x_12_3 = 1
x_13_1 = 1
x_14_7 = 1
x_15_5 = 1
x_16_1 = 1
x_17_4 = 1
x_18_6 = 1
x_19_3 = 1
x_20_4 = 1
x_21_6

# Wynik

Znalezione rozwiązanie jest jednym z rozwiązań dopuszczalnych. Możliwe jest uzyskanie innych rozwiązań dopuszczalnych w określonej liczbie (model.parameters.mip.pool), lecz nie da się nakazać solverowi aby znalazł wszystkie rozwiązania dopuszczalne gdy jest w trybie programowania matematycznego. (W tym celu należy użyć modelu programowania ograniczeń).